<a href="https://colab.research.google.com/github/abasu644/aiops/blob/master/Bert_Generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Init

In [1]:
!pip install torch
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 4.7MB/s 
     |████████████████████████████████| 645kB 52.1MB/s 


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForQuestionAnswering, BertForPreTraining ,BertAdam
from pytorch_pretrained_bert.modeling import BertForPreTraining, BertPreTrainedModel, BertModel, BertConfig,BertOnlyMLMHead

In [13]:
config=BertConfig.from_dict({'vocab_size':32000,'num_labels':4})
print(config)

{
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 4,
  "type_vocab_size": 2,
  "vocab_size": 32000
}



In [0]:
class BertForSequenceClassification2(BertPreTrainedModel):
    r"""
        **labels**: (`optional`) ``torch.LongTensor`` of shape ``(batch_size,)``:
            Labels for computing the sequence classification/regression loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
            If ``config.num_labels == 1`` a regression loss is computed (Mean-Square loss),
            If ``config.num_labels > 1`` a classification loss is computed (Cross-Entropy).

    Outputs: `Tuple` comprising various elements depending on the configuration (config) and inputs:
        **loss**: (`optional`, returned when ``labels`` is provided) ``torch.FloatTensor`` of shape ``(1,)``:
            Classification (or regression if config.num_labels==1) loss.
        **logits**: ``torch.FloatTensor`` of shape ``(batch_size, config.num_labels)``
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        **hidden_states**: (`optional`, returned when ``config.output_hidden_states=True``)
            list of ``torch.FloatTensor`` (one for the output of each layer + the output of the embeddings)
            of shape ``(batch_size, sequence_length, hidden_size)``:
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        **attentions**: (`optional`, returned when ``config.output_attentions=True``)
            list of ``torch.FloatTensor`` (one for each layer) of shape ``(batch_size, num_heads, sequence_length, sequence_length)``:
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.

    Examples::

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]

    """
    def __init__(self, config):
        super(BertForSequenceClassification2, self).__init__(config)
        config.num_labels = 2
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, self.config.num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None,
                position_ids=None, head_mask=None, labels=None):

        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids, 
                            head_mask=head_mask)

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)



In [0]:
modelpath = "bert-base-chinese"

model = BertForSequenceClassification2.from_pretrained(modelpath)


# Bert Generate One By One

In [0]:
input_text = "[CLS] I go to school by bus [SEP] "
target_text = "Ami scholey jachi"

In [33]:
# Load pre-trained model tokenizer (vocabulary)
modelpath = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(modelpath)
model = BertForMaskedLM2.from_pretrained(modelpath)

model.to('cuda')

example_pair = dict()

for i in range(0,len(target_text)+1):
  tokenized_text = tokenizer.tokenize(input_text)
  tokenized_text.extend(target_text[:i])
  tokenized_text.append('[MASK]')
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#   for _ in range(512-len(indexed_tokens)):
#     indexed_tokens.append(0)
  tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')
  
  loss_ids = [-1] * (len(tokenizer.convert_tokens_to_ids(tokenized_text))-1)
  if i == len(target_text):
    loss_ids.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize('[SEP]'))[0])
  else:
    loss_ids.append(tokenizer.convert_tokens_to_ids(target_text[i])[0])
#   for _ in range(512-len(loss_ids)):
#     loss_ids.append(-1)
  loss_tensors = torch.tensor([loss_ids]).to('cuda')
  
  example_pair[tokens_tensor] = loss_tensors
  print(tokenized_text,loss_ids,loss_ids[-1])
  print(len(indexed_tokens),len(loss_tensors))

TypeError: ignored

In [6]:
print(example_pair)

{tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,   103]],
       device='cuda:0'): tensor([[  -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1, 2769]],
       device='cuda:0'), tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,  2769,   103]],
       device='cuda:0'): tensor([[  -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1, 3022]],
       device='cuda:0'), tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,  2769,  3022,
           103]], device='cuda:0'): tensor([[  -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1, 1062]],
       device='cuda:0'), tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,  2769,  3022,
          1062,   103]], device='cuda:0'): tensor([[  -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1, 6722]],
       device='cuda:0'), tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,  2769,  3022,
          1062,  6722,   103]], device='cuda:0'): tensor([[ -1,  -1,  -1,  -1,  -1

In [3]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())

# hack to remove pooler, which is not used
# thus it produce None grad that break apex
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
optimizer = BertAdam(optimizer_grouped_parameters,
                             lr=5e-5,
                             warmup=0.1,
                             t_total=300)

NameError: ignored

In [8]:
model.train()
for i in range(0,60):
  eveloss = 0
  for k,v in example_pair.items():
    print(v.shape)
    loss = model(k,masked_lm_labels=v)
    eveloss += loss.mean().item()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  print("step "+ str(i) + " : " + str(eveloss))

torch.Size([1, 9])
torch.Size([1, 10])
torch.Size([1, 11])
torch.Size([1, 12])
torch.Size([1, 13])
torch.Size([1, 14])
torch.Size([1, 15])
step 0 : 41.22521352767944
torch.Size([1, 9])
torch.Size([1, 10])
torch.Size([1, 11])
torch.Size([1, 12])
torch.Size([1, 13])
torch.Size([1, 14])
torch.Size([1, 15])
step 1 : 26.684499740600586
torch.Size([1, 9])
torch.Size([1, 10])
torch.Size([1, 11])
torch.Size([1, 12])
torch.Size([1, 13])
torch.Size([1, 14])
torch.Size([1, 15])
step 2 : 23.901904106140137
torch.Size([1, 9])
torch.Size([1, 10])
torch.Size([1, 11])
torch.Size([1, 12])
torch.Size([1, 13])
torch.Size([1, 14])
torch.Size([1, 15])
step 3 : 17.304611206054688
torch.Size([1, 9])
torch.Size([1, 10])
torch.Size([1, 11])
torch.Size([1, 12])
torch.Size([1, 13])
torch.Size([1, 14])
torch.Size([1, 15])
step 4 : 28.499771118164062
torch.Size([1, 9])
torch.Size([1, 10])
torch.Size([1, 11])
torch.Size([1, 12])
torch.Size([1, 13])
torch.Size([1, 14])
torch.Size([1, 15])
step 5 : 7.966344833374023


Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.


step 42 : 3.0517578125e-05
torch.Size([1, 9])


Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.


torch.Size([1, 10])


...

torch.Size([1, 13])


Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.


torch.Size([1, 14])


Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.


torch.Size([1, 15])
step 59 : 3.814697265625e-06


# New Section

In [0]:
model.eval()
for k,v in example_pair.items():
    predictions = model(k)
    predicted_index = torch.argmax(predictions[0,len(predictions[0])-1]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    if '[SEP]' in predicted_token:
      break
    print(predicted_token)


['我']
['搭']
['公']
['車']
['上']
['學']


# Bert Generate One Time



In [0]:
input_text = "[CLS] I go to school by bus [SEP] "
target_text = "我搭公車上學"

In [0]:
# Load pre-trained model tokenizer (vocabulary)
modelpath = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(modelpath)
model = BertForMaskedLM.from_pretrained(modelpath)
model.to('cuda')

tokenized_text = tokenizer.tokenize(input_text)
for i in target_text:
  tokenized_text.append('[MASK]')
# tokenized_text.append('[SEP]')
for _ in range(128-len(tokenized_text)):
  tokenized_text.append('[MASK]')
# tokenized_text.append('[MASK]')
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')

loss_ids = []
loss_ids = [-1] * (len(tokenizer.tokenize(input_text)))
# loss_ids.extend(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(input_text)))
for i in target_text:
  loss_ids.append(tokenizer.convert_tokens_to_ids(i)[0])
loss_ids.append(tokenizer.convert_tokens_to_ids(['[SEP]'])[0])
for _ in range(128-len(loss_ids)):
  loss_ids.append(-1)
loss_tensors = torch.tensor([loss_ids]).to('cuda')

In [0]:
print(tokens_tensor,loss_tensors)
print(tokenizer.convert_ids_to_tokens(indexed_tokens))

tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,  

In [0]:
# # Prepare optimizer
# param_optimizer = list(model.named_parameters())

# # hack to remove pooler, which is not used
# # thus it produce None grad that break apex
# param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

# no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#         {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.05},
#         {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.01}
#         ]
# optimizer = BertAdam(optimizer_grouped_parameters,
#                              lr=5e-5)

# optimizer = torch.optim.Adam(model.parameters(), lr=5e-7)
# optimizer = torch.optim.SGD(model.parameters(), lr = 5e-5, momentum=0.9)
optimizer = torch.optim.Adamax(model.parameters(), lr = 5e-5)

In [0]:
model.train()
for i in range(0,300):
  loss = model(tokens_tensor,masked_lm_labels=loss_tensors)
  eveloss = loss.mean().item()
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  print("step "+ str(i) + " : " + str(eveloss))

step 0 : 7.913342475891113
step 1 : 7.833029270172119
step 2 : 7.197516441345215
step 3 : 6.325068473815918
step 4 : 5.70965576171875
step 5 : 5.5573506355285645
step 6 : 4.576362609863281
step 7 : 4.289361000061035
step 8 : 4.3420515060424805
step 9 : 4.592452526092529
step 10 : 4.3097662925720215
step 11 : 4.651139736175537
step 12 : 4.5207648277282715
step 13 : 4.477480411529541
step 14 : 4.2064900398254395
step 15 : 4.258938789367676
step 16 : 4.068099498748779
step 17 : 3.940248727798462
step 18 : 4.226219654083252
step 19 : 4.136264801025391
step 20 : 4.031257629394531
step 21 : 4.052146911621094
step 22 : 4.0309343338012695
step 23 : 3.9492135047912598
step 24 : 3.657867431640625
step 25 : 3.862555980682373
step 26 : 3.7650206089019775
step 27 : 3.5912110805511475
step 28 : 3.6331944465637207
step 29 : 3.9080066680908203
step 30 : 3.6006791591644287
step 31 : 3.616935968399048
step 32 : 3.6273269653320312
step 33 : 3.4186439514160156
step 34 : 3.361997365951538
step 35 : 3.23269

In [0]:
model.eval()
with torch.no_grad():
  predictions = model(tokens_tensor)
  start = len(tokenizer.tokenize(input_text))
  while start < len(predictions[0]):
    predicted_index = torch.argmax(predictions[0,start]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    if '[SEP]' in predicted_token:
        break
    print(predicted_token)
    start+=1

['我']
['搭']
['公']
['車']
['上']
['學']


# Bert Generate LSTM




In [0]:
input_text = "[CLS] I go to school by bus [SEP] "
target_text = "我搭公車上學"

In [0]:
# Load pre-trained model tokenizer (vocabulary)
modelpath = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(modelpath)
example_pair = dict()

for i in range(0,len(target_text)+1):
  tokenized_text = tokenizer.tokenize(input_text)
  tokenized_text.extend(target_text[:i])
  tokenized_text.append('[MASK]')
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')
  
  loss_ids = [-1] * (len(tokenizer.convert_tokens_to_ids(tokenized_text))-1)
  if i == len(target_text):
    loss_ids.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize('[SEP]'))[0])
  else:
    loss_ids.append(tokenizer.convert_tokens_to_ids(target_text[i])[0])
  loss_tensors = torch.tensor([loss_ids]).to('cuda')
  
  example_pair[tokens_tensor] = loss_tensors

In [0]:
class BertLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        modelpath = "bert-base-chinese"
        self.bert = BertModel.from_pretrained(modelpath)
        self.rnn = nn.LSTM(num_layers=2,dropout=0.2, input_size=768, hidden_size=768//2)
        self.fc = nn.Linear(384, self.bert.config.vocab_size)
        
    def forward(self, x,y=None):
        
        self.bert.train()
        encoded_layers, _ = self.bert(x)
        for i in encoded_layers:
          enc, _ = self.rnn(i[0,0])
        logits = self.fc(enc)
        if y is not None:
          loss_fct  = nn.CrossEntropyLoss(ignore_index=-1)
          loss = loss_fct(logits.view(-1, logits.shape[-1]), y.view(-1))
          return loss
        return logits

In [0]:
model = BertLSTM()
optimizer = torch.optim.Adamax(model.parameters(), lr = 5e-5)

In [0]:
model.to('cuda')
model.train()
for i in range(0,150):
  eveloss = 0
  for k,v in example_pair.items():
    optimizer.zero_grad()
    loss = model(k,v)
    eveloss += loss.mean().item()
    loss.backward()
    optimizer.step()
  print("step "+ str(i) + " : " + str(eveloss))

torch.Size([768])


RuntimeError: ignored

In [0]:
model.eval()
for k,v in example_pair.items():
    predictions = model(k)
    predicted_index = torch.argmax(predictions[0,-1]).item()
    if predicted_index < model.bert.config.vocab_size:
      predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    print(predicted_token)

['我']
['搭']
['公']
['車']
['上']
['學']
['[SEP]']
